# 有機分子の粘度計算スクリプト
### PubChem分子構造DL ⇒ 結合・角度・二面角・面外角生成 ⇒ ランダム配置による液体モデル作成 ⇒ LAMMPS粘度計算

### グローバル変数リスト
#### 関数：ExtractBondInfo
- *pair*: 結合リスト
- *trio*: 角度リスト
- *quad*: 二面角リスト
- *oop*: 面外角リスト

#### 関数：LoadParametersOPLS
- *ffname*: OPLS-AA力場に含まれるfftypeの番号
- *chg*: 電荷
- *mass*: 原子量
- *atom2bond*: fftypeの番号と結合の番号との対応
- *atom2angle*: fftypeの番号と角度の番号との対応
- *atom2dihedral*: fftypeの番号と二面角の番号との対応
- *atom2improper*: fftypeの番号と面外角の番号との対応
- *epsilon*: LJパラメータε 
- *sigma*: LJパラメータσ
- *bond_param*: 結合パラメータ
- *angle_param*: 角度パラメータ
- *dihedral_param*: 二面角パラメータ
- *improper_param*: 面外角パラメータ

#### 関数：ExtractBondType
- *elem_type*: 元素の種類数と内訳
- *atom_type*: fftypeの種類数と内訳
- *bond_type*: 結合の種類数と内訳
- *angle_type*: 角度の種類数と内訳
- *dihedral_type*: 二面角の種類数と内訳
- *improper_type*: 面外角の種類数と内訳
- *bond_prop*: 各結合の種類
- *angle_prop*: 各角度の種類 
- *dihedral_prop*: 各二面角の種類
- *improper_prop*: 各面外角の種類

#### fftype番号を各原子にアサイン
- *atom_prop*: 各原子に割り当てられたfftypeの番号

In [18]:
# pip install pubchemy

Could not fetch URL https://pypi.org/simple/pubchemy/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pubchemy/ (Caused by SSLError(SSLError("bad handshake: SysCallError(104, 'ECONNRESET')"))) - skipping
ERROR: Could not find a version that satisfies the requirement pubchemy (from versions: none)
ERROR: No matching distribution found for pubchemy
Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError(SSLError("bad handshake: SysCallError(104, 'ECONNRESET')"))) - skipping
Note: you may need to restart the kernel to use updated packages.


In [1]:
DEBUG = False

In [16]:
import os
import sys
import re
import numpy as np
import pandas as pd
#import pubchempy as pcp
from rdkit import rdBase, Chem
from rdkit.Chem import AllChem, Draw
from rdkit.Chem.Draw import IPythonConsole
pd.set_option ("display.max_columns", None)
if DEBUG == True: print ("rdkit version: ", rdBase.rdkitVersion)

### 各種関数の定義

In [4]:
def SetPrefix (prefix):
    b = prefix + "basic/"
    v = prefix + "visc/"
    t = prefix + "thermo/"
    if (os.path.exists (prefix) == False): os.mkdir (prefix)
    if (os.path.exists (b) == False): os.mkdir (b)
    if (os.path.exists (v) == False): os.mkdir (v)
    if (os.path.exists (t) == False): os.mkdir (t)
    return b, v, t

In [5]:
def LoadPubChemData (prefix, name):
    # PubChemからデータ取得    
    cid = pcp.get_cids (name, "name")[0]
    tmp = pcp.get_properties (["CanonicalSMILES", "IUPACName"], name, "name", as_dataframe = True)
    iupac = tmp["IUPACName"].iloc[-1]
    smiles = tmp["CanonicalSMILES"].iloc[-1]
    mol = Chem.MolFromSmiles (smiles)
    molH = Chem.AddHs (mol)
    AllChem.Compute2DCoords (molH)
    if DEBUG == True:
        print (smiles)
        print (cid)
        print (Chem.MolToMolBlock (molH, includeStereo = False))
        
    # 二次元構造の出力
    v = Chem.Draw.MolToImage (mol)
    v.save (prefix + str (iupac) + "_cid" + str (cid) + ".png")
    display (v)
    
    # 三次元構造の生成  
    AllChem.EmbedMolecule (molH, AllChem.ETKDG ())
    
    return mol, molH, cid, iupac

In [6]:
def ExtractBondInfo (mol):
    # pairリストの抽出
    for b in mol.GetBonds ():
        c = b.GetIdx ()
        i = b.GetBeginAtom ().GetIdx ()
        j = b.GetEndAtom ().GetIdx ()
        if DEBUG == True: print ("  {:2d} {:5d} {:5d}".format (c + 1, i + 1, j + 1))
        pair.append ([i, j])
        
    # trioリストの抽出
    c = 0
    for i in range (len (mol.GetAtoms ())):
        b = mol.GetAtomWithIdx (i).GetBonds ()
        if len (b) <= 1: continue
        for x in range (len (b)):
            a1 = b[x].GetBeginAtom ().GetIdx ()
            a2 = b[x].GetEndAtom ().GetIdx ()
            if i == a1: j = a2
            else: j = a1
            for y in range (x + 1, len (b), 1):
                if x == y: continue 
                a1 = b[y].GetBeginAtom ().GetIdx ()
                a2 = b[y].GetEndAtom ().GetIdx ()
                if i == a1: k = a2
                else: k = a1        
                if DEBUG == True: print ("  {:2d} {:5d} {:5d} {:5d}".format (c + 1, j + 1, i + 1, k + 1))
                trio.append ([j, i, k])
                c += 1     
                
    # quadリストの抽出    
    c = 0
    for b in mol.GetBonds ():
        i = b.GetBeginAtom ().GetIdx ()
        j = b.GetEndAtom ().GetIdx ()
        bi = mol.GetAtomWithIdx (i).GetBonds ()
        bj = mol.GetAtomWithIdx (j).GetBonds ()
        if len (bi) <= 1 or len (bj) <= 1: continue
        for x in range (len (bi)):
            a1 = bi[x].GetBeginAtom ().GetIdx ()
            a2 = bi[x].GetEndAtom ().GetIdx ()
            if b.GetIdx () == bi[x].GetIdx (): continue
            if i == a1: k = a2
            else: k = a1    
            for y in range (len (bj)):
                a1 = bj[y].GetBeginAtom ().GetIdx ()
                a2 = bj[y].GetEndAtom ().GetIdx ()
                if b.GetIdx () == bj[y].GetIdx (): continue
                if j == a1: l = a2
                else: l = a1    
                if DEBUG == True: print ("  {:2d} {:5d} {:5d} {:5d} {:5d}".format (c + 1, k + 1, i + 1, j + 1, l + 1))
                quad.append ([k, i, j, l])
                c += 1     

    # oopリストの抽出        
    c = 0
    for i in range (len (mol.GetAtoms ())):
        bi = mol.GetAtomWithIdx (i).GetBonds ()
        hi = mol.GetAtomWithIdx (i).GetHybridization ()
        si = mol.GetAtomWithIdx (i).GetSymbol ()
        if str (hi) != "SP2": continue    
        if str (si) != "C": continue
        link = [bi[x].GetBeginAtom ().GetIdx () for x in range (len (bi)) if i != bi[x].GetBeginAtom ().GetIdx ()]
        link.extend ([bi[x].GetEndAtom ().GetIdx () for x in range (len (bi)) if i != bi[x].GetEndAtom ().GetIdx ()])
        j = 100000
        for l in link:
            hl = mol.GetAtomWithIdx (l).GetHybridization ()
            if str (hl) == "SP2":
                if l < j: j = l
        link = [l for l in link if l != j] 
        s0 = str (mol.GetAtomWithIdx (link[0]).GetSymbol ())
        s1 = str (mol.GetAtomWithIdx (link[1]).GetSymbol ())
        if (s0 == "H") and (s1 != "H"):
            k = link[1]
            m = link[0]
        elif (s0 != "H") and (s1 == "H"):
            k = link[0]
            m = link[1]
        else:
            if link[0] < link[1]:
                k = link[0]
                m = link[1]
            else:
                k = link[1]
                m = link[0]     
        if DEBUG == True: print ("  {:2d} {:5d} {:5d} {:5d} {:5d}".format (c + 1, j + 1, i + 1, k + 1, m + 1))
        oop.append ([j, i, k, m])
        c i= 1

In [7]:
def LoadParametersOPLS (disp):   
    # fftypeと電荷の読み込み
    start = 113
    end = 962
    with open ("./oplsaa.lt", "r") as fp:
        for n, d in enumerate (fp):
            if n < start or n > end: continue
            x = re.split ("[@:#\n]", d)
            y = x[2].split ()
            num = int (y[0])
            chg[num] = float (y[2])
            ffname[num] = x[3]
            if DEBUG == True: print ("[{:4d}]    #{:3d}  {:30s} (CHG: {:8.5f})".format (n - start, num, ffname[num], chg[num]))        

    # 原子量の読み込み        
    start = 1023
    end = 1872
    with open ("oplsaa.lt", "r") as fp:
        for n, d in enumerate (fp):
            if n < start or n > end: continue
            x = re.split ("[@:#\n]", d)
            y = x[2].split ()
            num = int (y[0])
            mass[num] = y[1]
            if DEBUG == True: print ("[{:4d}]    #{:3d}  (MASS: {:10.5f})".format (n - start, num, mass[num]))
            
    # fftypeの番号と結合・角度・二面角・面外角番号との対応読み込み
    start = 1943
    end = 2792
    with open ("oplsaa.lt", "r") as fp:
        for n, d in enumerate (fp):
            if n < start or n > end: continue
            x = re.split ("[badi_}@:#\n]", d)
            num = int (x[7])
            atom2bond[num]     = int (x[9])
            atom2angle[num]    = int (x[11])
            atom2dihedral[num] = int (x[13])
            atom2improper[num] = int (x[15])
            if DEBUG == True: print ("[{:4d}]    #{:3d}  b{:3d}  a{:3d}  d{:3d}  i{:3d}".format (n - start, num, atom2bond[num], atom2angle[num], atom2dihedral[num], atom2improper[num]))     

    # 分散力パラメータの読み込み
    start = 2859
    end = 3708
    with open ("oplsaa.lt", "r") as fp:
        for n, d in enumerate (fp):
            if n < start or n > end: continue
            x = re.split ("[_@:#\n]", d)
            y = x[13].split ()
            num = int (x[3])
            epsilon[num] = y[1]
            sigma[num] = y[2]
            if DEBUG == True: print ("[{:4d}]    #{:3d}  LJ - epsilon: {:10.5f}  sigma: {:10.5f}".format (n - start, num, epsilon[num], sigma[num]))

    # 結合パラメータの読み込み
    start = 3719
    end = 4097
    with open ("oplsaa.lt", "r") as fp:
        for n, d in enumerate (fp):
            if n < start or n > end: continue
            x = re.split ("[_@:#\n]", d)
            y = x[4].split ()
            num = n - start
            bond_param[num][0] = float (x[3])
            bond_param[num][1] = float (y[0])
            bond_param[num][2] = float (y[1])
            bond_param[num][3] = float (y[2])
            if DEBUG == True: print ("[{:4d}]    #{:3d} - {:3d}  bond - k: {:10.5f}  r: {:10.5f}".format (num, int (bond_param[num][0]), int (bond_param[num][1]), bond_param[num][2], bond_param[num][3]))                

    # 角度パラメータの読み込み
    start = 4495
    end = 5511
    with open ("oplsaa.lt", "r") as fp:
        for n, d in enumerate (fp):
            if n < start or n > end: continue
            x = re.split ("[_@:#\n]", d)
            y = x[5].split ()
            num = n - start
            angle_param[num][0] = float (x[3])
            angle_param[num][1] = float (x[4])
            angle_param[num][2] = float (y[0])
            angle_param[num][3] = float (y[1])
            angle_param[num][4] = float (y[2])
            if DEBUG == True: print ("[{:4d}]    #{:3d} - {:3d} - {:3d}  angle - k: {:10.5f}  theta: {:10.5f}".format (num, int (angle_param[num][0]), int (angle_param[num][1]), int (angle_param[num][2]), angle_param[num][3], angle_param[num][4]))
                
    # 二面角パラメータの読み込み                
    start = 6547
    end = 7346
    with open ("oplsaa.lt", "r") as fp:
        for n, d in enumerate (fp):
            if n < start or n > end: continue
            x = re.split ("[_@:#\n]", d)
            y = x[6].split ()
            num = n - start
            if x[3] == "X": x[3] = "000"
            if y[0] == "X": y[0] = "000"
            dihedral_param[num][0] = float (x[3])
            dihedral_param[num][1] = float (x[4])
            dihedral_param[num][2] = float (x[5])
            dihedral_param[num][3] = float (y[0])
            dihedral_param[num][4] = float (y[1])
            dihedral_param[num][5] = float (y[2])
            dihedral_param[num][6] = float (y[3])
            dihedral_param[num][7] = float (y[4])
            if DEBUG == True: print ("[{:4d}]    #{:3d} - {:3d} - {:3d} - {:3d}  dihedral - k1: {:10.5f}  k2: {:10.5f}  k3: {:10.5f}  k4: {:10.5f}".format (num, int (dihedral_param[num][0]), int (dihedral_param[num][1]), int (dihedral_param[num][2]), int (dihedral_param[num][3]), dihedral_param[num][4], dihedral_param[num][5], dihedral_param[num][6], dihedral_param[num][7]))                

    # 面外角パラメータの読み込み
    specified = [8169, 8173, 8177, 8181, 8185, 8189]
    with open ("oplsaa.lt", "r") as fp:
        num = 0
        for n, d in enumerate (fp):
            for i in range (len (specified)):
                if n != specified[i]: continue
                x = re.split ("[_@:#\n]", d)
                y = x[6].split ()
                if x[3] == "X": x[3] = "000"
                if x[4] == "X": x[4] = "000"
                if x[5] == "X": x[5] = "000"
                if y[0] == "X": y[0] = "000"
                improper_param[num][0] = float (y[0])
                improper_param[num][1] = float (x[5])
                improper_param[num][2] = float (x[4])
                improper_param[num][3] = float (x[3])
                improper_param[num][4] = float (y[1])
                improper_param[num][5] = float (y[2])
                if DEBUG == True: print ("[{:4d}]    #{:3d} - {:3d} - {:3d} - {:3d}  improper - k: {:10.5f}  chi: {:10.5f}".format (num, int (improper_param[num][0]), int (improper_param[num][1]), int (improper_param[num][2]), int (improper_param[num][3]), improper_param[num][4], improper_param[num][5]))
                num += 1

    # リストの表示
    if disp == True:
        for n in range (57, 907, 1):
            print ("TYPE# {:3d}  WT: {:7.3f}  CHG: {:6.3f}  {}".format (n, mass[n], chg[n], ffname[n]))

In [8]:
def ExtractBondType (mol):   
    # 元素の種類数と内訳
    for a in mol.GetAtoms ():
        i = a.GetIdx ()
        n = a.GetAtomicNum ()        
        if i == 0:
            elem_type.append (n)
            continue
        err = 0
        for j in range (len (elem_type)):
            if elem_type[j] == n: err = err + 1
        if err == 0: elem_type.append (n)
    elem_type.sort ()
    if DEBUG == True:
        print ("Number of elements: {}".format (len (elem_type)))
        print (elem_type)

    # fftypeの種類数と内訳
    for i in range (len (atom_prop)):
        if i == 0:
            atom_type.append (atom_prop[i])
            continue
        err = 0
        for j in range (len (atom_type)):
            if atom_type[j] == atom_prop[i]: err = err + 1
        if err == 0: atom_type.append (atom_prop[i])
    atom_type.sort ()
    if DEBUG == True:
        print ("Number of atom types: {}".format (len (atom_type)))
        print (atom_type)
    
    # 結合の種類数と内訳
    for i in range (len (pair)):
        p = [atom_prop[pair[i][0]], atom_prop[pair[i][1]]]
        if i == 0:
            bond_type.append (p)
            continue
        err = 0
        for j in range (len (bond_type)):
            if bond_type[j][0] == p[0] and bond_type[j][1] == p[1]: err = err + 1
            if bond_type[j][0] == p[1] and bond_type[j][1] == p[0]: err = err + 1                  
        if err == 0: bond_type.append (p)   
    if DEBUG == True:
        print ("Number of bond types: {}".format (len (bond_type)))
        print (bond_type)
    for i in range (len (pair)):
        p = [atom_prop[pair[i][0]], atom_prop[pair[i][1]]]
        for j in range (len (bond_type)):
            if (bond_type[j][0] == p[0] and bond_type[j][1] == p[1]) or (bond_type[j][0] == p[1] and bond_type[j][1] == p[0]):
                bond_prop[i] = j
        if DEBUG == True:
            print ("Bond#: {:4d}   Type#: {:2d}    Pair#: {:4d}({:3d})-{:4d}({:3d})".format (i + 1, bond_prop[i] + 1, pair[i][0] + 1, bond_type[bond_prop[i]][0], pair[i][1] + 1, bond_type[bond_prop[i]][1]))
            
    # 角度の種類数と内訳
    for i in range (len (trio)):
        p = [atom_prop[trio[i][0]], atom_prop[trio[i][1]], atom_prop[trio[i][2]]]
        if i == 0:
            angle_type.append (p)
            continue
        err = 0
        for j in range (len (angle_type)):
            if angle_type[j][1] == p[1]:
                if angle_type[j][0] == p[0] and angle_type[j][2] == p[2]: err = err + 1
                if angle_type[j][0] == p[2] and angle_type[j][2] == p[0]: err = err + 1                  
        if err == 0: angle_type.append (p)  
    if DEBUG == True:
        print ("Number of angle types: {}".format (len (angle_type)))
        print (angle_type)
    for i in range (len (trio)):
        p = [atom_prop[trio[i][0]], atom_prop[trio[i][1]], atom_prop[trio[i][2]]]
        for j in range (len (angle_type)):  
            if (angle_type[j][1] == p[1]):
                if (angle_type[j][0] == p[0] and angle_type[j][2] == p[2]) or (angle_type[j][0] == p[2] and angle_type[j][2] == p[0]):
                    angle_prop[i] = j
        if DEBUG == True:
            print ("Angle#: {:4d}   Type#: {:2d}    Trio#: {:4d}({:3d})-{:4d}({:3d})-{:4d}({:3d})".format (i + 1, angle_prop[i] + 1, trio[i][0] + 1, angle_type[angle_prop[i]][0], trio[i][1] + 1, angle_type[angle_prop[i]][1], trio[i][2] + 1, angle_type[angle_prop[i]][2]))
    
    # 二面角の種類数と内訳
    for i in range (len (quad)):
        p = [atom_prop[quad[i][0]], atom_prop[quad[i][1]], atom_prop[quad[i][2]], atom_prop[quad[i][3]]]
        if i == 0:
            dihedral_type.append (p)
            continue
        err = 0
        for j in range (len (dihedral_type)):
            if dihedral_type[j][0] == p[0] and dihedral_type[j][1] == p[1] and dihedral_type[j][2] == p[2] and dihedral_type[j][3] == p[3]: err = err + 1
            if dihedral_type[j][0] == p[3] and dihedral_type[j][1] == p[2] and dihedral_type[j][2] == p[1] and dihedral_type[j][3] == p[0]: err = err + 1
        if err == 0: dihedral_type.append (p)  
    if DEBUG == True:
        print ("Number of dihedral types: {}".format (len (dihedral_type)))
        print (dihedral_type)
    for i in range (len (quad)):
        p = [atom_prop[quad[i][0]], atom_prop[quad[i][1]], atom_prop[quad[i][2]], atom_prop[quad[i][3]]]
        for j in range (len (dihedral_type)):  
            if (dihedral_type[j][0] == p[0] and dihedral_type[j][1] == p[1] and dihedral_type[j][2] == p[2] and dihedral_type[j][3] == p[3]) or (dihedral_type[j][0] == p[3] and dihedral_type[j][1] == p[2] and dihedral_type[j][2] == p[1] and dihedral_type[j][3] == p[0]):
                dihedral_prop[i] = j
        if DEBUG == True:
            print ("Dihedral#: {:4d}   Type#: {:2d}    Quad#: {:4d}({:3d})-{:4d}({:3d})-{:4d}({:3d})-{:4d}({:3d})".format (i + 1, dihedral_prop[i] + 1, quad[i][0] + 1, dihedral_type[dihedral_prop[i]][0], quad[i][1] + 1, dihedral_type[dihedral_prop[i]][1], quad[i][2] + 1, dihedral_type[dihedral_prop[i]][2], quad[i][3] + 1, dihedral_type[dihedral_prop[i]][3]))

    # 面外角の種類数と内訳
    for i in range (len (oop)):
        p = [atom_prop[oop[i][0]], atom_prop[oop[i][1]], atom_prop[oop[i][2]], atom_prop[oop[i][3]]]
        if i == 0:
            improper_type.append (p)
            continue
        err = 0
        for j in range (len (improper_type)):
            if improper_type[j][0] == p[0] and improper_type[j][1] == p[1] and improper_type[j][2] == p[2] and improper_type[j][3] == p[3]: err = err + 1
            if improper_type[j][0] == p[2] and improper_type[j][1] == p[1] and improper_type[j][2] == p[0] and improper_type[j][3] == p[3]: err = err + 1
            if improper_type[j][0] == p[0] and improper_type[j][1] == p[1] and improper_type[j][2] == p[3] and improper_type[j][3] == p[2]: err = err + 1
            if improper_type[j][0] == p[2] and improper_type[j][1] == p[1] and improper_type[j][2] == p[3] and improper_type[j][3] == p[0]: err = err + 1
            if err == 0: improper_type.append (p)    
    if DEBUG == True:
        print ("Number of improper types: {}".format (len (improper_type)))
        print (improper_type)
    for i in range (len (oop)):
        p = [atom_prop[oop[i][0]], atom_prop[oop[i][1]], atom_prop[oop[i][2]], atom_prop[oop[i][3]]]
        for j in range (len (improper_type)):  
            if (improper_type[j][0] == p[0] and improper_type[j][1] == p[1] and improper_type[j][2] == p[2] and improper_type[j][3] == p[3]) or (improper_type[j][0] == p[2] and improper_type[j][1] == p[1] and improper_type[j][2] == p[0] and improper_type[j][3] == p[3]) or (improper_type[j][0] == p[0] and improper_type[j][1] == p[1] and improper_type[j][2] == p[3] and improper_type[j][3] == p[2]) or (improper_type[j][0] == p[2] and improper_type[j][1] == p[1] and improper_type[j][2] == p[3] and improper_type[j][3] == p[0]):
                improper_prop[i] = j
        if DEBUG == True:
            print ("Improper#: {:4d}   Type#: {:2d}    OOP#: {:4d}({:3d})-{:4d}({:3d})-{:4d}({:3d})-{:4d}({:3d})".format (i + 1, improper_prop[i] + 1, oop[i][0] + 1, improper_type[improper_prop[i]][0], oop[i][1] + 1, improper_type[improper_prop[i]][1], oop[i][2] + 1, improper_type[improper_prop[i]][2], oop[i][3] + 1, improper_type[improper_prop[i]][3])) 

In [9]:
def ShowAtomNumber (mol, label): # label = "atomNote" or "atomLabel" or "molAtomMapNumber"
    for a in mol.GetAtoms ():
        a.SetProp (label, str (a.GetIdx ()))
    return mol

def CheckTotalCharge (mol, mode):
    s = 0.0
    for a in mol.GetAtoms ():
        i = a.GetIdx ()
        if mode == 0:
            if a.GetSymbol () == "C": atom_prop[i] = 81
            elif a.GetSymbol () == "H": atom_prop[i] = 85
            elif a.GetSymbol () == "O": atom_prop[i] = 96
        print ("# {:<4d}{:2s}  BND: {:1d}  FF: {:3d}  WT: {:7.3f}  CHG: {:6.3f}".format (i + 1, a.GetSymbol (), len (a.GetBonds ()), atom_prop[i], a.GetMass(), chg[atom_prop[i]]))
        s = s + chg[atom_prop[i]]
    print ("Sum of atomic charge: {:8.5e}".format (s))
    if s > 1.0e-4: print ("***** ERROR IN ATOMIC CHARGE *****")   

def SetKnownCompounds (name):
    if name == "ethanol":
        atom_prop[0] = 80
        atom_prop[1] = 99
        atom_prop[8] = 97
    elif name == "benzene":
        atom_prop[0:6] = 90
        atom_prop[6:12] = 91
    elif name == "dodecane":
        atom_prop[0] = 80
        atom_prop[11] = 80
    elif name == "1-hexadecene":
        atom_prop[0] = 80
        atom_prop[14] = 87
        atom_prop[15] = 88
        atom_prop[45] = 89
        atom_prop[46] = 89
        atom_prop[47] = 89
    elif name == "alpha-pinene":
        atom_prop[0] = 80
        atom_prop[8] = 80
        atom_prop[9] = 80
        atom_prop[4] = 82
        atom_prop[6] = 82
        atom_prop[7] = 84
        atom_prop[1] = 86
        atom_prop[2] = 87
        atom_prop[13] = 89
    elif name == "verbenone":
        atom_prop[0] = 80
        atom_prop[9] = 80
        atom_prop[10] = 80
        atom_prop[5] = 82
        atom_prop[7] = 82
        atom_prop[8] = 84
        atom_prop[1] = 86
        atom_prop[2] = 87
        atom_prop[14] = 89
        atom_prop[3] = 222
        atom_prop[4] = 223
    elif name == "geranyl acetate":
        atom_prop[0] = 80
        atom_prop[11] = 80
        atom_prop[12] = 80
        atom_prop[13] = 80
        atom_prop[1] = 86
        atom_prop[2] = 87
        atom_prop[17] = 89
        atom_prop[5] = 86
        atom_prop[6] = 87
        atom_prop[22] = 89
        atom_prop[7] = 431
        atom_prop[23] = 410
        atom_prop[24] = 410
        atom_prop[8] = 408
        atom_prop[9] = 406
        atom_prop[10] = 407

In [10]:
def SetMassCenter (mol):
    p = mol.GetConformer ().GetPositions ()
    mw = 0.0
    mx = 0.0
    my = 0.0
    mz = 0.0
    for a in mol.GetAtoms ():
        i = a.GetIdx ()
        n = atom_prop[i]
        mw += mass[n]
        mx += mass[n] * p[i, 0]
        my += mass[n] * p[i, 1]
        mz += mass[n] * p[i, 2]
    mx /= mw
    my /= mw
    mz /= mw
    p[:, :] -= [mx, my, mz]
    return p

def RotationMatrix (ang):
    rotx = np.array ([[1.0, 0.0, 0.0],
                      [0.0, np.cos (ang[0]), -np.sin (ang[0])],
                      [0.0, np.sin (ang[0]), np.cos (ang[0])]])
    roty = np.array ([[np.cos (ang[1]), 0.0, np.sin (ang[1])],
                      [0.0, 1.0, 0.0],
                      [-np.sin (ang[1]), 0.0, np.cos (ang[1])]])
    rotz = np.array ([[np.cos (ang[2]), -np.sin (ang[2]), 0.0],
                      [np.sin (ang[2]), np.cos (ang[2]), 0.0],
                      [0.0, 0.0, 1.0]])
    return rotz.dot (roty).dot (rotx)

def CheckOverlap (n, org, pos, lx, ly, lz, rcut):
    err = 0
    for i in range (len (pos)):
        for m in range (len (org)):
            for j in range (len (pos)):
                rx = (pos[i, 0] - org[m, j, 0]) / lx
                ry = (pos[i, 1] - org[m, j, 1]) / ly
                rz = (pos[i, 2] - org[m, j, 2]) / lz
                while rx < -0.5: rx += 1.0
                while rx >  0.5: rx -= 1.0
                while ry < -0.5: ry += 1.0
                while ry >  0.5: ry -= 1.0                
                while rz < -0.5: rz += 1.0
                while rz >  0.5: rz -= 1.0       
                rx *= lx
                ry *= ly
                rz *= lz
                rr = np.sqrt (rx * rx + ry * ry + rz * rz)
                if rr < rcut:
                    err += 1
                    return err
    return err

In [11]:
def WriteStructureFile (molenum, mol, lx, ly, lz, bulk, prefix, filename):
    # 元素種のソート番号
    
    sort = np.zeros (len (mol.GetAtoms ()), dtype = int)
    for a in mol.GetAtoms ():
        i = a.GetIdx ()
        n = atom_prop[i]
        for k in range (len (atom_type)):
            if n == atom_type[k]: sort[i] = k + 1

    # ファイルの出力
    with open (prefix + filename + ".lammps05", "w") as fp:
        # ヘッダー
        fp.writelines ("LAMMPS 2005 data file for {}\n\n".format (filename))
        # 原子・結合・角度・二面角・面外角の数
        fp.writelines ("{:7d} atoms\n".format (molenum * len (mol.GetAtoms ())))
        if len (pair) > 0: fp.writelines ("{:7d} bonds\n".format (molenum * len (pair)))
        if len (trio) > 0: fp.writelines ("{:7d} angles\n".format (molenum * len (trio)))
        if len (quad) > 0: fp.writelines ("{:7d} dihedrals\n".format (molenum * len (quad)))
        if len (oop)  > 0: fp.writelines ("{:7d} impropers\n".format (molenum * len (oop)))
        fp.writelines ("\n")
        # 原子・結合・角度・二面角・面外角の種類数
        fp.writelines ("{:7d} atom types\n".format (len (atom_type)))
        if len (bond_type) > 0: fp.writelines ("{:7d} bond types\n".format (len (bond_type)))
        if len (angle_type) > 0: fp.writelines ("{:7d} angle types\n".format (len (angle_type)))  
        if len (dihedral_type) > 0: fp.writelines ("{:7d} dihedral types\n".format (len (dihedral_type)))
        if len (improper_type) > 0: fp.writelines ("{:7d} improper types\n".format (len (improper_type)))
        fp.writelines ("\n")
        # セル長
        fp.writelines ("{:16.9f} {:16.9f} xlo xhi\n".format (0.0, lx))
        fp.writelines ("{:16.9f} {:16.9f} ylo yhi\n".format (0.0, ly))
        fp.writelines ("{:16.9f} {:16.9f} zlo zhi\n\n".format (0.0, lz))
        # 原子量
        fp.writelines ("Masses\n\n")
        n = 0
        for i in atom_type:
            fp.writelines ("{:7d} {:15.6f}     # {:30s}\n".format (n + 1, mass[i], ffname[i]))
            n += 1
        # 分散力パラメータ
        fp.writelines ("\nPair Coeffs\n\n")
        n = 0
        for i in atom_type:
            fp.writelines ("{:7d} {:15.6f} {:15.6f}     # {:30s}\n".format (n + 1, epsilon[i], sigma[i], ffname[i]))
            n += 1
        # 結合パラメータ
        if len (pair) > 0:
            fp.writelines ("\nBond Coeffs\n\n")
            for i in range (len (bond_type)):
                a1 = atom2bond[bond_type[i][0]]
                a2 = atom2bond[bond_type[i][1]]
                for n in range (len (bond_param)):
                    b1 = int (bond_param[n][0])
                    b2 = int (bond_param[n][1])
                    if (a1 == b1 and a2 == b2) or (a1 == b2 and a2 == b1):
                        k = bond_param[n][2]
                        r = bond_param[n][3]
                        break
                fp.writelines ("{:7d} {:15.6f} {:15.6f}     # Atom type {:3d} - {:3d}\n".format (i + 1, k, r, bond_type[i][0], bond_type[i][1]))           
        # 角度パラメータ
        if len (trio) > 0:
            fp.writelines ("\nAngle Coeffs\n\n")
            for i in range (len (angle_type)):
                a1 = atom2angle[angle_type[i][0]]
                a2 = atom2angle[angle_type[i][1]]
                a3 = atom2angle[angle_type[i][2]]
                for n in range (len (angle_param)):
                    b1 = int (angle_param[n][0])
                    b2 = int (angle_param[n][1])
                    b3 = int (angle_param[n][2])
                    if (a1 == b1 and a3 == b3) or (a1 == b3 and a3 == b1):
                        k = angle_param[n][3]
                        t = angle_param[n][4]
                        break
                fp.writelines ("{:7d} {:15.6f} {:15.6f}     # Atom type {:3d} - {:3d} - {:3d}\n".format (i + 1, k, t, angle_type[i][0], angle_type[i][1], angle_type[i][2]))        
        # 二面角パラメータ
        if len (quad) > 0:
            fp.writelines ("\nDihedral Coeffs\n\n")
            for i in range (len (dihedral_type)):
                a1 = atom2dihedral[dihedral_type[i][0]]
                a2 = atom2dihedral[dihedral_type[i][1]]
                a3 = atom2dihedral[dihedral_type[i][2]]
                a4 = atom2dihedral[dihedral_type[i][3]]
                flag = 0
                for n in range (len (dihedral_param)):
                    b1 = int (dihedral_param[n][0])
                    b2 = int (dihedral_param[n][1])
                    b3 = int (dihedral_param[n][2])
                    b4 = int (dihedral_param[n][3])
                    if   (b1 == 0) and (a2 == b2 and a3 == b3 and a4 == b4): flag = 1
                    elif (b4 == 0) and (a1 == b1 and a2 == b2 and a3 == b3): flag = 1
                    elif (b1 == 0 and b4 == 0) and ((a2 == b2 and a3 == b3) or (a2 == b3 and a3 == b2)): flag = 1
                    elif (a1 == b1 and a2 == b2 and a3 == b3 and a4 == b4) or (a1 == b4 and a2 == b3 and a3 == b2 and a4 == b1): flag = 1
                    if flag != 0:
                        k1 = dihedral_param[n][4]
                        k2 = dihedral_param[n][5]
                        k3 = dihedral_param[n][6]
                        k4 = dihedral_param[n][7]
                        break
                fp.writelines ("{:7d} {:15.6f} {:15.6f} {:15.6f} {:15.6f}    # Atom type {:3d} - {:3d} - {:3d} - {:3d}\n".format (i + 1, k1, k2, k3, k4, dihedral_type[i][0], dihedral_type[i][1], dihedral_type[i][2], dihedral_type[i][3]))        
        # 面外角パラメータ   
        if len (oop) > 0:
            fp.writelines ("\nImproper Coeffs\n\n")
            for i in range (len (improper_type)):
                a1 = atom2improper[improper_type[i][0]]
                a2 = atom2improper[improper_type[i][1]]
                a3 = atom2improper[improper_type[i][2]]
                a4 = atom2improper[improper_type[i][3]]
                flag = 0
                for n in range (len (improper_param)):
                    b1 = int (improper_param[n][0])
                    b2 = int (improper_param[n][1])
                    b3 = int (improper_param[n][2])
                    b4 = int (improper_param[n][3])
                    if b1 == 0 and b2 == a2 and b3 == 0 and b4 == 0: flag = 1
                    elif (b2 == a2 and b3 == 0 and b4 == 0) and (b1 == a1 or b1 == a3): flag = 1
                    elif (b2 == 0) and (b1 == a1 and b3 == a3 and b4 == a4): flag = 1                
                    if flag != 0:
                        k = improper_param[n][4]
                        c = improper_param[n][5]
                        break
                fp.writelines ("{:7d} {:15.6f} {:15.6f}    # Atom type {:3d} - {:3d} - {:3d} - {:3d}\n".format (i + 1, k, c, improper_type[i][0], improper_type[i][1], improper_type[i][2], improper_type[i][3]))        
        # 原子                    
        fp.writelines ("\nAtoms\n\n")
        for m in range (molenum):
            j = m * len (mol.GetAtoms ())
            for a in mol.GetAtoms ():
                i = a.GetIdx ()
                n = atom_prop[i]                
                fp.writelines ("{:7d} {:6d} {:6d} {:9.6f} {:15.9f} {:15.9f} {:15.9f}\n".format (j + (i + 1), m + 1, sort[i], chg[n], pos[m, i, 0], pos[m, i, 1], pos[m, i, 2]))
        # 結合
        if (len (pair) > 0): 
            fp.writelines ("\nBonds\n\n")
            for m in range (molenum):
                j = m * len (mol.GetAtoms ())
                k = m * len (pair)
                for i in range (len (pair)):
                    a1 = bond_type[bond_prop[i]][0]
                    a2 = bond_type[bond_prop[i]][1]                    
                    fp.writelines ("{:7d} {:6d} {:6d} {:6d}     # Atom type {:3d} - {:3d}\n".format (k + (i + 1), bond_prop[i] + 1, j + pair[i][0] + 1, j + pair[i][1] + 1, a1, a2))        
        # 角度        
        if (len (trio) > 0): 
            fp.writelines ("\nAngles\n\n")
            for m in range (molenum):
                j = m * len (mol.GetAtoms ())
                k = m * len (trio)
                for i in range (len (trio)):
                    a1 = angle_type[angle_prop[i]][0]
                    a2 = angle_type[angle_prop[i]][1]        
                    a3 = angle_type[angle_prop[i]][2]             
                    fp.writelines ("{:7d} {:6d} {:6d} {:6d} {:6d}     # Atom type {:3d} - {:3d} - {:3d}\n".format (k + (i + 1), angle_prop[i] + 1, j + trio[i][0] + 1, j + trio[i][1] + 1, j + trio[i][2] + 1, a1, a2, a3))
        # 二面角        
        if (len (quad) > 0):
            fp.writelines ("\nDihedrals\n\n")
            for m in range (molenum):
                j = m * len (mol.GetAtoms ())
                k = m * len (quad)
                for i in range (len (quad)):
                    a1 = dihedral_type[dihedral_prop[i]][0]
                    a2 = dihedral_type[dihedral_prop[i]][1]        
                    a3 = dihedral_type[dihedral_prop[i]][2]
                    a4 = dihedral_type[dihedral_prop[i]][3]  
                    fp.writelines ("{:7d} {:6d} {:6d} {:6d} {:6d} {:6d}     # Atom type {:3d} - {:3d} - {:3d} - {:3d}\n".format (k + (i + 1), dihedral_prop[i] + 1, j + quad[i][0] + 1, j + quad[i][1] + 1, j + quad[i][2] + 1, j + quad[i][3] + 1, a1, a2, a3, a4))
        # 面外角
        if (len (oop) > 0):
            fp.writelines ("\nImpropers\n\n")
            for m in range (molenum):
                j = m * len (mol.GetAtoms ())
                k = m * len (oop)
                for i in range (len (oop)):
                    a1 = improper_type[improper_prop[i]][0]
                    a2 = improper_type[improper_prop[i]][1]        
                    a3 = improper_type[improper_prop[i]][2]
                    a4 = improper_type[improper_prop[i]][3]  
                    fp.writelines ("{:7d} {:6d} {:6d} {:6d} {:6d} {:6d}     # Atom type {:3d} - {:3d} - {:3d} - {:3d}\n".format (k + (i + 1), improper_prop[i] + 1, j + oop[i][0] + 1, j + oop[i][1] + 1, j + oop[i][2] + 1, j + oop[i][3] + 1, a1, a2, a3, a4))

In [12]:
def WriteInputFile(
    molenum,
    lx,
    ly,
    lz,
    duration,
    interval,
    prefix,
    filename,
    rcut=1.8,
    seed=99,
    delta_t=0.2,
    temperature=293.15,
    pressure=0.0,
    rand_num=12345,
    cut_max=10.0,
):
    # カットオフ半径
    mn = lx
    if ly < mn:
        mn = ly
    if lz < mn:
        mn = lz
    cut1 = 0.5 * mn
    cut2 = 0.5 * mn
    if cut1 > cut_max:
        cut1 = cut_max
    if cut2 > cut_max:
        cut2 = cut_max

    # ファイル出力
    with open(prefix + filename + ".in", "w") as fp:
        # -----INITIALIZE
        fp.writelines("#-----INITIALIZE\n")
        fp.writelines(
            "clear\nunits real\ndimension 3\nnewton on\nboundary p p p\natom_style full\natom_modify map array\n\n"
        )
        # -----OPLS-AA FORCE-FIELD
        fp.writelines("#-----OPLS-AA FORCE-FIELD\n")
        fp.writelines("pair_style lj/cut/coul/long {:.1f} {:.1f}\n".format(cut1, cut2))
        fp.writelines("bond_style harmonic\n")
        fp.writelines("angle_style harmonic\n")
        fp.writelines("dihedral_style opls\n")
        fp.writelines("improper_style harmonic\n")
        fp.writelines("kspace_style pppm 1.0e-4\n\n")
        # -----MOLECULAR STRUCTURE
        fp.writelines("#-----MOLECULAR STRUCTURE\n")
        fp.writelines("read_data {}.lammps05\n\n".format(filename))
        # -----NEIGHBOR LIST
        fp.writelines("#-----NEIGHBOR LIST\n")
        fp.writelines("neighbor 2.0 bin\n")
        fp.writelines("neigh_modify every 10 delay 0 check yes\n\n")
        # -----DISPLAY
        fp.writelines("#-----DISPLAY\n")
        fp.writelines("thermo 100\n")
        if molenum == 1:
            fp.writelines("thermo_style custom step temp ke pe\n\n")
        else:
            fp.writelines("thermo_style custom step temp ke pe lx ly lz density\n\n")
        # -----INITIAL VELOCITY AND DELTA T
        fp.writelines("#-----INITIAL VELOCITY AND DELTA T\n")
        fp.writelines("velocity all create {:.2f} {}\n".format(temperature, rand_num))
        fp.writelines("run_style verlet\n")
        fp.writelines("timestep {:.1f}\n\n".format(delta_t))
        # -----NPT-MD (COMPRESS)
        if molenum > 1:
            fp.writelines("#-----NPT-MD (COMPRESS)\n")
            fp.writelines(
                "dump 1 all atom {} {}_pres.traj\n".format(interval, filename)
            )
            fp.writelines(
                "fix 1 all npt temp {:.2f} {:.2f} 1 iso {:.2f} {:.2f} 100\n".format(
                    temperature, temperature, 1000.0, 1000.0
                )
            )
            fp.writelines("run {}\n".format(duration))
            fp.writelines("unfix 1\n")
            fp.writelines("undump 1\n\n")
        # -----NPT-MD (RELAX)
        if molenum > 1:
            fp.writelines("#-----NPT-MD (RELAX)\n")
            fp.writelines(
                "dump 1 all atom {} {}_relax.traj\n".format(interval, filename)
            )
            fp.writelines(
                "fix 1 all npt temp {:.2f} {:.2f} 1 iso {:.2f} {:.2f} 100\n".format(
                    temperature, temperature, pressure, pressure
                )
            )
            fp.writelines("run {}\n".format(duration))
            fp.writelines("unfix 1\n")
            fp.writelines("undump 1\n\n")
        # -----NVT-MD
        fp.writelines("#-----NVT-MD\n")
        fp.writelines("dump 1 all atom {} {}_nvt.traj\n".format(interval, filename))
        fp.writelines(
            "fix 1 all nvt temp {:.2f} {:.2f} 1\n".format(temperature, temperature)
        )
        if molenum > 1:
            fp.writelines("reset_timestep 0\n")
        fp.writelines("run {}\n".format(duration))


In [13]:
def WriteViscInputFile(
    lx,
    ly,
    lz,
    prefix,
    filename,
    rcut=1.8,
    seed=99,
    delta_t=0.2,
    temperature=293.15,
    pressure=0.0,
    rand_num=12345,
    cut_max=10.0,
):
    # カットオフ半径
    mn = lx
    if ly < mn:
        mn = ly
    if lz < mn:
        mn = lz
    cut1 = 0.5 * mn
    cut2 = 0.5 * mn
    if cut1 > cut_max:
        cut1 = cut_max
    if cut2 > cut_max:
        cut2 = cut_max

    # ファイル出力
    with open(prefix + filename + ".in", "w") as fp:
        # -----SYSTEM SETTING
        fp.writelines("#-----SYSTEM SETTING\n")
        fp.writelines("units real\n")
        fp.writelines("atom_style full\n")
        fp.writelines("bond_style harmonic\n")
        fp.writelines("angle_style harmonic\n")
        fp.writelines("dihedral_style opls\n")
        fp.writelines("improper_style harmonic\n")
        fp.writelines("pair_style lj/cut/coul/long {:.1f} {:.1f}\n".format(cut1, cut2))
        fp.writelines("pair_modify mix geometric\n")
        fp.writelines("special_bonds lj/coul 0.0 0.0 0.5\n")
        fp.writelines("kspace_style pppm 1.0e-4\n\n")

        # -----MOLECULAR STRUCTURE
        fp.writelines("#-----MOLECULAR STRUCTURE\n")
        fp.writelines("read_data {}.lammps05\n\n".format(filename))

        # -----NEIGHBOR LIST
        fp.writelines("#-----NEIGHBOR LIST\n")
        fp.writelines("neighbor 4.0 bin\n")
        fp.writelines("neigh_modify delay 5 every 1\n\n")

        # -----DISPLAY
        fp.writelines("#-----DISPLAY\n")
        fp.writelines("thermo 1000\n")
        fp.writelines("thermo_style custom step temp ke pe lx ly lz press density\n\n")

        # -----INITIAL VELOCITY AND DELTA T
        fp.writelines("#-----INITIAL VELOCITY AND DELTA T\n")
        fp.writelines("variable dt equal {:.2f}\n".format(delta_t))
        fp.writelines("variable temp equal {:.2f}\n".format(temperature))
        fp.writelines("timestep ${dt}\n")
        fp.writelines(
            "velocity all create ${temp} "
            + "{:} mom yes dist gaussian\n\n".format(rand_num)
        )

        # -----COMPRESS SYSTEM BY NPT UNDER HIGH PRESSURE
        fp.writelines("#-----COMPRESS SYSTEM BY NPT UNDER HIGH PRESSURE\n")
        fp.writelines(
            "fix 1 all npt temp ${temp} ${temp} 100.0 iso 1000.0 1000.0 1000.0\n"
        )
        fp.writelines("fix 2 all momentum 100 linear 1 1 1 rescale\n")
        fp.writelines("run 50000\n")
        fp.writelines("unfix 1\n\n")

        # -----CHANGE SYSTEM PRESSURE
        fp.writelines("#-----CHANGE SYSTEM PRESSURE\n")
        fp.writelines(
            "fix 1 all npt temp ${temp} ${temp} 100.0 iso 1000.0 1.0 1000.0\n"
        )
        fp.writelines("run 50000\n")
        fp.writelines("unfix 1\n\n")

        # -----RELAXATION BY NPT UNDER ATMOSPHERIC PRESSURE
        fp.writelines("#-----RELAXATION BY NPT UNDER ATMOSPHERIC PRESSURE\n")
        fp.writelines("fix 1 all npt temp ${temp} ${temp} 100.0 iso 1.0 1.0 100.0\n")
        fp.writelines("run 500000\n")
        fp.writelines("unfix 1\n\n")

        # -----RNEMD SETIING
        fp.writelines("#-----RNEMD SETTING\n")
        fp.writelines("reset_timestep 0\n")
        fp.writelines("variable dt equal 1.00\n")
        fp.writelines("timestep ${dt}\n")
        fp.writelines("write_restart visc.restart\n")
        fp.writelines("fix 1 all npt temp ${temp} ${temp} 100.0 iso 1.0 1.0 1000.0\n\n")

        # -----EQUILIBRIUM RNEMD RUN
        fp.writelines("#-----EQUILIBRIUM RNEMD RUN\n")
        fp.writelines("fix 4 all viscosity 100 x z 20\n")
        fp.writelines(
            "compute layers all chunk/atom bin/1d z center 0.05 units reduced\n"
        )
        fp.writelines("fix 5 all ave/chunk 20 50 1000 layers vx\n")
        fp.writelines("variable dVx equal f_5[11][3]-f_5[1][3]\n")
        fp.writelines("thermo_style custom step temp ke pe press density f_4 v_dVx\n")
        fp.writelines("run 100000\n")
        fp.writelines("unfix 4\n")
        fp.writelines("uncompute layers\n")
        fp.writelines("unfix 5\n\n")

        # -----DATA GATHERING RNEMD RUN
        fp.writelines("#-----DATA GATHERING RNEMD RUN\n")
        fp.writelines("reset_timestep 0\n")
        fp.writelines("fix 4 all viscosity 100 x z 20\n")
        fp.writelines(
            "compute layers all chunk/atom bin/1d z center 0.05 units reduced\n"
        )
        fp.writelines(
            "fix 5 all ave/chunk 20 50 1000 layers vx file rnemd_profile.dat\n"
        )
        fp.writelines("variable mom equal f_4\n")
        fp.writelines("variable time equal step*${dt}\n")
        fp.writelines("variable area equal lx*ly\n")
        fp.writelines("variable mylz equal lz\n")
        fp.writelines("variable myd equal density\n")
        fp.writelines(
            "fix 6 all ave/time 100 10 1000 v_time v_mom v_area v_mylz v_myd file rnemd_momenta.dat\n"
        )
        fp.writelines("dump 1 all atom 1000 rnemd_trajectory.dat\n")
        fp.writelines("run 500000\n")


In [14]:
def WriteThermoInputFile(
    lx,
    ly,
    lz,
    prefix,
    filename,
    rcut=1.8,
    seed=99,
    delta_t=0.2,
    temperature=293.15,
    pressure=0.0,
    rand_num=12345,
    cut_max=10.0,
):
    # カットオフ半径
    mn = lx
    if ly < mn:
        mn = ly
    if lz < mn:
        mn = lz
    cut1 = 0.5 * mn
    cut2 = 0.5 * mn
    if cut1 > cut_max:
        cut1 = cut_max
    if cut2 > cut_max:
        cut2 = cut_max

    # ファイル出力
    with open(prefix + filename + ".in", "w") as fp:
        # -----SYSTEM SETTING
        fp.writelines("#-----SYSTEM SETTING\n")
        fp.writelines("units real\n")
        fp.writelines("atom_style full\n")
        fp.writelines("bond_style harmonic\n")
        fp.writelines("angle_style harmonic\n")
        fp.writelines("dihedral_style opls\n")
        fp.writelines("improper_style harmonic\n")
        fp.writelines("pair_style lj/cut/coul/long {:.1f} {:.1f}\n".format(cut1, cut2))
        fp.writelines("pair_modify mix geometric\n")
        fp.writelines("special_bonds lj/coul 0.0 0.0 0.5\n")
        fp.writelines("kspace_style pppm 1.0e-4\n\n")

        # -----MOLECULAR STRUCTURE
        fp.writelines("#-----MOLECULAR STRUCTURE\n")
        fp.writelines("read_data {}.lammps05\n\n".format(filename))

        # -----NEIGHBOR LIST
        fp.writelines("#-----NEIGHBOR LIST\n")
        fp.writelines("neighbor 4.0 bin\n")
        fp.writelines("neigh_modify delay 5 every 1\n\n")

        # -----DISPLAY
        fp.writelines("#-----DISPLAY\n")
        fp.writelines("thermo 1000\n")
        fp.writelines("thermo_style custom step temp ke pe lx ly lz press density\n\n")

        # -----INITIAL VELOCITY AND DELTA T
        fp.writelines("#-----INITIAL VELOCITY AND DELTA T\n")
        fp.writelines("variable dt equal {:.2f}\n".format(delta_t))
        fp.writelines("variable temp equal {:.2f}\n".format(temperature))
        fp.writelines("timestep ${dt}\n")
        fp.writelines(
            "velocity all create ${temp} "
            + "{:} mom yes dist gaussian\n\n".format(rand_num)
        )

        # -----COMPRESS SYSTEM BY NPT UNDER HIGH PRESSURE
        fp.writelines("#-----COMPRESS SYSTEM BY NPT UNDER HIGH PRESSURE\n")
        fp.writelines(
            "fix 1 all npt temp ${temp} ${temp} 100.0 iso 1000.0 1000.0 1000.0\n"
        )
        fp.writelines("fix 2 all momentum 100 linear 1 1 1 rescale\n")
        fp.writelines("run 50000\n")
        fp.writelines("unfix 1\n\n")

        # -----CHANGE SYSTEM PRESSURE
        fp.writelines("#-----CHANGE SYSTEM PRESSURE\n")
        fp.writelines(
            "fix 1 all npt temp ${temp} ${temp} 100.0 iso 1000.0 1.0 1000.0\n"
        )
        fp.writelines("run 50000\n")
        fp.writelines("unfix 1\n\n")

        # -----RELAXATION BY NPT UNDER ATMOSPHERIC PRESSURE
        fp.writelines("#-----RELAXATION BY NPT UNDER ATMOSPHERIC PRESSURE\n")
        fp.writelines("fix 1 all npt temp ${temp} ${temp} 100.0 iso 1.0 1.0 100.0\n")
        fp.writelines("run 500000\n")
        fp.writelines("unfix 1\n\n")

        # -----RNEMD SETIING
        fp.writelines("#-----RNEMD SETTING\n")
        fp.writelines("reset_timestep 0\n")
        fp.writelines("variable dt equal 1.00\n")
        fp.writelines("timestep ${dt}\n")
        fp.writelines("write_restart thermo.restart\n")
        fp.writelines("compute ke all ke/atom\n")
        fp.writelines("variable atmT atom c_ke/1.5\n")
        fp.writelines("fix 1 all npt temp ${temp} ${temp} 100.0 iso 1.0 1.0 1000.0\n\n")

        # -----EQUILIBRIUM RNEMD RUN
        fp.writelines("#-----EQUILIBRIUM RNEMD RUN\n")
        fp.writelines("fix 4 all thermal/conductivity 100 z 20\n")
        fp.writelines(
            "compute layers all chunk/atom bin/1d z lower 0.05 units reduced\n"
        )
        fp.writelines("fix 5 all ave/chunk 20 50 1000 layers v_atmT\n")
        fp.writelines("variable dTx equal f_5[11][3]-f_5[1][3]\n")
        fp.writelines("thermo_style custom step temp ke pe press density f_4 v_dTx\n")
        fp.writelines("run 100000\n")
        fp.writelines("unfix 4\n")
        fp.writelines("uncompute layers\n")
        fp.writelines("unfix 5\n\n")

        # -----DATA GATHERING RNEMD RUN
        fp.writelines("#-----DATA GATHERING RNEMD RUN\n")
        fp.writelines("reset_timestep 0\n")
        fp.writelines("fix 4 all thermal/conductivity 100 z 20\n")
        fp.writelines(
            "compute layers all chunk/atom bin/1d z lower 0.05 units reduced\n"
        )
        fp.writelines(
            "fix 5 all ave/chunk 20 50 1000 layers v_atmT file rnemd_profile.dat\n"
        )
        fp.writelines("variable ene equal f_4\n")
        fp.writelines("variable time equal step*${dt}\n")
        fp.writelines("variable area equal lx*ly\n")
        fp.writelines("variable mylz equal lz\n")
        fp.writelines("variable myd equal density\n")
        fp.writelines(
            "fix 6 all ave/time 100 10 1000 v_time v_ene v_area v_mylz v_myd file rnemd_energy.dat\n"
        )
        fp.writelines("dump 1 all atom 1000 rnemd_trajectory.dat\n")
        fp.writelines("run 500000\n")

# <font color="Green">～PubChemからLAMMPSへ繋ぐメインパート～</font>

### LAMMPS計算のフォルダ準備

In [17]:
def main():
    comp_name = "7,7-Dimethyl-2-propyl-bicyclo[3.1.1]hept-2-ene"
    prefix    = "/work02/onodera/03_EVcoolant/" + comp_name + "/"
    prefix_basic, prefix_visc, prefix_thermo = SetPrefix (prefix)

    ### PubChemから化合物データの読み込み
    mol, molH, cid, iupac = LoadPubChemData (prefix, comp_name)
    
    ### 結合・角度・二面角・面外角の生成
    pair = []
    trio = []
    quad = []
    oop  = []
    ExtractBondInfo (molH)
    
    ### OPLS-AA力場の読み込み
    ffname         = np.full (907, "012345678901234567890123456789")
    chg            = np.zeros (907)
    mass           = np.zeros (907)
    atom2bond      = np.zeros (907, dtype = int)
    atom2angle     = np.zeros (907, dtype = int)
    atom2dihedral  = np.zeros (907, dtype = int)
    atom2improper  = np.zeros (907, dtype = int)
    epsilon        = np.zeros (907) 
    sigma          = np.zeros (907)
    bond_param     = np.zeros ((379, 4))
    angle_param    = np.zeros ((1017, 5))
    dihedral_param = np.zeros ((800, 8))
    improper_param = np.zeros ((6, 6))
    LoadParametersOPLS (disp = False)
    
    ### 各原子へfftype番号のアサイン
    # ※デフォルトでは、Cはメチレン、Hはアルキル、Oはアルコール
    atom_prop = np.zeros (len (molH.GetAtoms ()), dtype = int)
    CheckTotalCharge (molH, 0)
    
    ### fftype番号の手動修正・再アサイン
    ShowAtomNumber (molH, "atomNote")
    for n in range (57, 907, 1):
        print ("TYPE# {:3d}  WT: {:7.3f}  CHG: {:6.3f}  {}".format (n, mass[n], chg[n], ffname[n]))
        
    ######  <font color="Red"> *atom_prop*[i]にfftype番号を手動入力する</font>
    SetKnownCompounds (comp_name)
    atom_prop[10] = 80
    atom_prop[11] = 80
    atom_prop[0] = 80
    atom_prop[9] = 84
    atom_prop[6] = 82
    atom_prop[8] = 82

    atom_prop[3] = 86
    atom_prop[4] = 87

    atom_prop[19] = 89
    CheckTotalCharge (molH, 1)
    
    ### 元素・結合・角度・二面角・面外角種類数の決定
    elem_type     = []
    atom_type     = []
    bond_type     = []
    angle_type    = []
    dihedral_type = []
    improper_type = []
    bond_prop     = np.zeros (len (pair), dtype = int)
    angle_prop    = np.zeros (len (trio), dtype = int) 
    dihedral_prop = np.zeros (len (quad), dtype = int)
    improper_prop = np.zeros (len (oop),  dtype = int)
    ExtractBondType (molH)
    
    run_lammps(iupac=iupac, cid=cid, molH=molH, 
               prefix_basic=prefix_basic, prefix_visc=prefix_visc, prefix_thermo=prefix_thermo)

PermissionError: [Errno 13] Permission denied: '/work02/onodera/03_EVcoolant/7,7-Dimethyl-2-propyl-bicyclo[3.1.1]hept-2-ene/'

   ### LAMMPSによるMDシミュレーション

#### モデル作成
  - *lxyz*: セル長 [A]
  - *nmol*: 分子数
  - *rcut*: 分子同士の最小許容接近距離 [A]
  - *seed*: 分子のランダム配置のための乱数シード
  
#### MD計算条件
  - *delta_t*: 積分時間 [fs]
  - *temperature*: 温度 [K]
  - *pressure*: 圧力 [atm]
  - *rand_num*: 初期速度のMaxwell-Boltzmann分布計算のための乱数シード
  - *cut_max*: カットオフ半径の最大値 [A]

In [38]:
def run_lammps(
    iupac=None,
    cid=None,
    molH=None,
    prefix_basic=None,
    prefix_visc=None,
    prefix_thermo=None,
    lxyz=40.0,
    nmol=100,
    rcut=1.8,
    seed=99,
    delta_t=0.2,
    temperature=293.15,
    pressure=0.0,
    rand_num=12345,
    cut_max=10.0,
    filename='in',
):
    #### 孤立モデルの作成
    if iupac is not None:
        filename = str(iupac)
    if cid is not None:
        filename += "_cid" + str(cid)
    
    iso = SetMassCenter(molH)
    pos = np.zeros((1, len(iso), 3))
    pos[0, :, :] = iso[:, :] + [0.5 * lxyz, 0.5 * lxyz, 0.5 * lxyz]
    WriteStructureFile(1, molH, lxyz, lxyz, lxyz, pos, prefix_basic, filename)
    WriteInputFile(1, lxyz, lxyz, lxyz, 10000, 100, prefix_basic, filename)

    #### 液体モデルの作成（粘度）
    v = np.power(lxyz, 3.0)
    lx = np.power((v / 3.0), (1.0 / 3.0))
    ly = lx
    lz = lx * 3.0
    print("BOX SIZE: {:.1f} x {:.1f} x {:.1f}".format(lx, ly, lz))

    bulk = np.zeros((nmol, len(iso), 3))
    np.random.seed(seed=seed)

    # 1分子目の配置
    n = 0
    print(
        "MOL #%5d/%-5d POS(%6.1f %6.1f %6.1f) ROT(%6.1f %6.1f %6.1f) TRY (%4d)"
        % (n + 1, nmol, 0.5 * lx, 0.5 * ly, 0.5 * lz, 0.0, 0.0, 0.0, 0)
    )
    bulk[n, :, :] = iso[:, :] + [0.5 * lx, 0.5 * ly, 0.5 * lz]

    # 乱数を用いて分子の座標を計算
    n = 1
    c = 1
    while n < nmol:
        pos = np.zeros((len(iso), 3))
        cen = np.random.random(3)
        cen[0] *= lx
        cen[1] *= ly
        cen[2] *= lz
        ang = np.random.random(3) * np.pi
        mat = RotationMatrix(ang)
        for i in range(len(iso)):
            pos[i] = mat.dot(iso[i]) + cen
        err = CheckOverlap(n, bulk[0:n], pos, lx, ly, lz, rcut)
        if err == 0:
            print(
                "MOL #%5d/%-5d POS(%6.1f %6.1f %6.1f) ROT(%6.1f %6.1f %6.1f) TRY (%4d)"
                % (
                    n + 1,
                    nmol,
                    cen[0],
                    cen[1],
                    cen[2],
                    np.rad2deg(ang[0]),
                    np.rad2deg(ang[1]),
                    np.rad2deg(ang[2]),
                    c,
                )
            )
            bulk[n, :, :] = pos[:, :]
            c = 1
            n += 1
        else:
            c += 2
    print(
        "Totally {} molecules ({} atoms) were arranged in {:.0f} x {:.0f} x {:.0f} box.".format(
            nmol, nmol * len(molH.GetAtoms()), lx, ly, lz
        )
    )
    #### 液体モデルの作成（熱伝導率）
    filename = str(iupac) + "_visc"
    WriteStructureFile(nmol, molH, lx, ly, lz, bulk, prefix_visc, filename)
    WriteViscInputFile(lx, ly, lz, prefix_visc, filename)

    lx = lxyz
    ly = lx
    lz = lx
    print("BOX SIZE: {:.1f} x {:.1f} x {:.1f}".format(lx, ly, lz))

    bulk = np.zeros((nmol, len(iso), 3))
    np.random.seed(seed=seed)

    # 1分子目の配置
    n = 0
    print(
        "MOL #%5d/%-5d POS(%6.1f %6.1f %6.1f) ROT(%6.1f %6.1f %6.1f) TRY (%4d)"
        % (n + 1, nmol, 0.5 * lx, 0.5 * ly, 0.5 * lz, 0.0, 0.0, 0.0, 0)
    )
    bulk[n, :, :] = iso[:, :] + [0.5 * lx, 0.5 * ly, 0.5 * lz]

    # 乱数を用いて分子の座標を計算
    n = 1
    c = 1
    while n < nmol:
        pos = np.zeros((len(iso), 3))
        cen = np.random.random(3)
        cen[0] *= lx
        cen[1] *= ly
        cen[2] *= lz
        ang = np.random.random(3) * np.pi
        mat = RotationMatrix(ang)
        for i in range(len(iso)):
            pos[i] = mat.dot(iso[i]) + cen
        err = CheckOverlap(n, bulk[0:n], pos, lx, ly, lz, rcu)
        if err == 0:
            print(
                "MOL #%5d/%-5d POS(%6.1f %6.1f %6.1f) ROT(%6.1f %6.1f %6.1f) TRY (%4d)"
                % (
                    n + 1,
                    nmol,
                    cen[0],
                    cen[1],
                    cen[2],
                    np.rad2deg(ang[0]),
                    np.rad2deg(ang[1]),
                    np.rad2deg(ang[2]),
                    c,
                )
            )
            bulk[n, :, :] = pos[:, :]
            c = 1
            n += 1
        else:
            c += 1
    print(
        "Totally {} molecules ({} atoms) were arranged in {:.0f} x {:.0f} x {:.0f} box.".format(
            nmol, nmol * len(molH.GetAtoms()), lx, ly, lz
        )
    )

    filename = str(iupac) + "_thermo"
    WriteStructureFile(nmol, molH, lx, ly, lz, bulk, prefix_thermo, filename)
    WriteThermoInputFile(lx, ly, lz, prefix_thermo, filename)

    filename = str(iupac) + "_bulk"
    WriteStructureFile(nmol, molH, lx, ly, lz, bulk, prefix_basic, filename)
    WriteInputFile(nmol, lx, ly, lz, 200000, 100, prefix_basic, filename)

